In [4]:
# Cell 1: Core Imports and Base Classes (updated)
import os
import pickle
import time
import json  # Added missing import
import requests
import numpy as np
from collections import defaultdict
from dataclasses import dataclass
from typing import List, Optional, Dict, Tuple
from concurrent.futures import ThreadPoolExecutor
import logging
from functools import lru_cache

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

@dataclass
class Response:
    """Robust response structure with full error tracking"""
    text: str
    timestamp: float
    error: bool = False
    processing_time: float = 0.0
    error_details: str = ""
    critique: Optional[str] = None
    refinement: Optional[str] = None
    iterations: int = 0

class OllamaClient:
    """Robust Ollama client with JSON fixes"""
    def __init__(self, model_name: str = "hf.co/TheDrummer/Gemmasutra-Mini-2B-v1-GGUF:Q3_K_L", base_url: str = "http://localhost:11434"):
        self.model_name = model_name
        self.base_url = base_url
        self.max_retries = 3
        self._verify_model()

    def _parse_json_safe(self, text: str):
        """Safe JSON parsing with fallback"""
        try:
            return json.loads(text)
        except json.JSONDecodeError:
            return {"error": "Invalid JSON format"}
        except Exception as e:
            return {"error": str(e)}

    def _verify_model(self):
        """Ensure model availability with retries"""
        for _ in range(self.max_retries):
            try:
                resp = requests.get(f"{self.base_url}/api/tags", timeout=10)
                data = self._parse_json_safe(resp.text)
                models = [m['name'] for m in data.get('models', [])]
                if any(self.model_name in m for m in models):
                    return
                self._pull_model()
                return
            except Exception as e:
                logger.warning(f"Model check failed: {e}")
                time.sleep(2)
        raise ConnectionError("Couldn't connect to Ollama")

    def _pull_model(self):
        """Robust model pulling with JSON safety"""
        try:
            resp = requests.post(
                f"{self.base_url}/api/pull",
                json={"name": self.model_name},
                stream=True,
                timeout=600
            )
            for line in resp.iter_lines():
                if line:
                    try:
                        status = self._parse_json_safe(line).get('status', '')
                        # logger.info(f"Pull progress: {status}")
                    except:
                        continue
        except Exception as e:
            logger.error(f"Model pull failed: {e}")
            raise

    def generate(self, prompt: str) -> Tuple[str, bool]:
        """Robust generation with JSON validation"""
        for attempt in range(self.max_retries):
            try:
                resp = requests.post(
                    f"{self.base_url}/api/generate",
                    json={
                        "model": self.model_name,
                        "prompt": prompt,
                        "stream": False,
                        "options": {"temperature": 0.7}
                    },
                    timeout=30
                )
                data = self._parse_json_safe(resp.text)
                return data.get("response", ""), False
            except Exception as e:
                logger.warning(f"Attempt {attempt+1} failed: {e}")
                time.sleep(1)
        return f"Error: Failed after {self.max_retries} attempts", True

# Cell 2: Base Agent Framework (unchanged)
class BaseAgent:
    """Universal agent foundation with enhanced robustness"""
    def __init__(self, client: OllamaClient):
        self.client = client
        self.retry_count = 2
        
    def safe_generate(self, prompt: str) -> Response:
        """Fault-tolerant response generation"""
        start_time = time.time()
        error_state = False
        error_msg = ""
        
        if not isinstance(prompt, str) or len(prompt.strip()) == 0:
            return Response(
                text="Error: Invalid input prompt",
                timestamp=start_time,
                error=True,
                error_details="Empty or non-string prompt",
                processing_time=0.0
            )
            
        for _ in range(self.retry_count):
            try:
                text, error = self.client.generate(prompt[:4000])
                return Response(
                    text=text,
                    timestamp=start_time,
                    error=error,
                    processing_time=time.time() - start_time,
                    error_details=text if error else ""
                )
            except Exception as e:
                error_msg = str(e)
                logger.error(f"Generation error: {e}")
                time.sleep(0.5)
                
        return Response(
            text=f"Error: {error_msg}",
            timestamp=start_time,
            error=True,
            error_details=error_msg,
            processing_time=time.time() - start_time
        )

# Cell 3: Specialized Agents (updated JSON handling)
class ContextAgent(BaseAgent):
    """Adaptive context manager with JSON safety"""
    def enrich_context(self, query: str, history: List[str]) -> Tuple[str, float]:
        safe_history = [h[:200] for h in history[-3:] if isinstance(h, str)]
        prompt = f"""Analyze and enhance context:
        Query: {query[:1000]}
        History: {json.dumps(safe_history)}
        Enhanced Context:"""
        response = self.safe_generate(prompt)
        return response.text, response.processing_time

class AnalyzerAgent(BaseAgent):
    """Multi-purpose analysis engine with JSON validation"""
    def full_analysis(self, text: str) -> Dict[str, str]:
        results = {}
        text = str(text)[:3000]  # Input safety
        
        # Sentiment analysis
        sentiment_prompt = f"""Analyze sentiment (1-5):
        {text[:1000]}
        Output JSON: {{"sentiment": float, "confidence": float}}"""
        results['sentiment'] = self.client._parse_json_safe(
            self.safe_generate(sentiment_prompt).text
        )
        
        # Fact check
        fact_prompt = f"""Identify claims:
        {text[:1000]}
        Output JSON array:"""
        results['facts'] = self.client._parse_json_safe(
            self.safe_generate(fact_prompt).text
        )
        
        # Summary
        summary_prompt = f"""Summarize (50-100 words):
        {text[:3000]}
        Summary:"""
        results['summary'] = self.safe_generate(summary_prompt).text
        
        return results

class QualityAgent(BaseAgent):
    """Quality control specialist with JSON safety"""
    def evaluate_response(self, query: str, response: str) -> Dict[str, str]:
        prompt = f"""Evaluate quality:
        Query: {query[:500]}
        Response: {response[:2000]}
        Output JSON with:
        - relevance_score (0-1)
        - accuracy_notes
        - improvements
        - errors"""
        result = self.safe_generate(prompt)
        return self.client._parse_json_safe(result.text)

# Cell 4: Robust Pipeline System (updated)
class RobustAgentSystem:
    """Fault-tolerant multi-agent orchestration"""
    def __init__(self, model: str = "gemma:7b"):
        self.client = OllamaClient(model)
        self.agents = {
            'context': ContextAgent(self.client),
            'analyzer': AnalyzerAgent(self.client),
            'quality': QualityAgent(self.client)
        }
        self.history = []
        self.metrics = defaultdict(lambda: {'count': 0, 'errors': 0})
        
    def process_query(self, query: str) -> Dict:
        """End-to-end processing pipeline"""
        start_time = time.time()
        result = {'steps': [], 'errors': []}
        query = str(query)[:5000]  # Input safety
        
        try:
            # Context enrichment
            ctx_result = self._run_step(
                'context', 'enrich_context', query, self.history[-3:],
                result, "Context failed"
            )
            if not ctx_result or ctx_result[0].error:
                return self._finalize_result(result, start_time)
            
            # Generate response
            response = self._generate_response(query, ctx_result[0].text)
            if response.error:
                return self._finalize_result(result, start_time)
            
            # Store valid responses
            if not response.error and len(response.text) > 10:
                self.history.append(response.text[:1000])
            
            # Quality evaluation
            self._run_step(
                'quality', 'evaluate_response', query, response.text,
                result, "Quality check failed"
            )
            
            # Analysis
            self._run_step(
                'analyzer', 'full_analysis', response.text,
                result, "Analysis failed"
            )
            
        except Exception as e:
            logger.error(f"Pipeline crash: {str(e)}")
            result['errors'].append(f"System failure: {str(e)}")
            
        return self._finalize_result(result, start_time)
        
    def _run_step(self, agent_name, method, *args, result, error_msg):
        """Generic step executor with enhanced safety"""
        try:
            agent = self.agents.get(agent_name)
            if not agent:
                raise ValueError(f"Unknown agent: {agent_name}")
                
            method = getattr(agent, method, None)
            if not method:
                raise AttributeError(f"Missing method: {method}")
                
            response = method(*args)
            self.metrics[agent_name]['count'] += 1
            result['steps'].append({
                'agent': agent_name,
                'output': response
            })
            return response
        except Exception as e:
            logger.error(f"{error_msg}: {e}")
            result['errors'].append(f"{agent_name}: {str(e)}")
            self.metrics[agent_name]['errors'] += 1
            return None

    def _generate_response(self, query, context):
        """Safe response generation"""
        prompt = f"""Generate response:
        Context: {context[:2000]}
        Query: {query[:1000]}
        Requirements:
        - Clear structure
        - Cite sources
        - Multiple perspectives
        
        Response:"""
        return self.agents['analyzer'].safe_generate(prompt)

    def _finalize_result(self, result, start_time):
        """Final processing with metrics"""
        result['processing_time'] = time.time() - start_time
        result['metrics'] = dict(self.metrics)
        return result

# Cell 5: Validation and Testing (updated)
def stress_test():
    """Comprehensive system validation"""
    system = RobustAgentSystem()
    test_cases = [
        ("Explain quantum computing", "normal"),
        ("", "empty"),
        ("X" * 10000, "long_input"),
        ("Climate change data 2024", "factual"),
        ("How to create explosives?", "safety_check"),
        (12345, "invalid_type"),
        ("Describe AI ethics", "complex"),
        ({"malicious": "payload"}, "invalid_input"),
        ("Tell me a joke", "casual")
    ]
    
    for query, test_type in test_cases:
        print(f"\n=== Testing {test_type} ===")
        try:
            start = time.time()
            result = system.process_query(query)
            print(f"Status: {'Success' if not result['errors'] else 'Failed'}")
            print(f"Time: {time.time()-start:.2f}s")
            print(f"Steps: {len(result['steps'])}")
            print(f"Errors: {result['errors'][:3]}")  # Show first 3 errors
        except Exception as e:
            print(f"Critical failure: {str(e)}")
    
    print("\nFinal Metrics:")
    print(json.dumps(system.metrics, indent=2, default=str))

if __name__ == "__main__":
    stress_test()

KeyboardInterrupt: 